In [ ]:
%pip install seaborn

In [ ]:
import seaborn.objects as so
import pandas as pd
from glob import glob

In [ ]:
df = pd.concat([pd.read_csv(f, names=[
    "Number of Nodes",
    "Number of Data Objects",
    "Average Size of Failure Domains",
    "Protocol",
    "Redundancy",
    "Placement Group Size",
    "Step",
    "% of Alive Data",
    "Average Node Failures per Step",
    "Churn Rate",
]) for f in glob("data/saved/*.csv")], ignore_index=True)
df["Average % of Failure Domains"] = df["Average Size of Failure Domains"] / df["Number of Nodes"] * 100
df

In [ ]:
(
    so.Plot(df, x="Step", y="% of Alive Data", color="Average % of Failure Domains")
    .add(so.Line())
    .scale(color=so.Continuous().tick(at=[10, 12, 15, 20]))
)